<a href="https://colab.research.google.com/github/hyojgun/hyojeong/blob/main/BERT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")
text = "Transformers Is so COOL"

encoded = tokenizer(text)
print(encoded)

input_ids = encoded["input_ids"]
print(input_ids)

decoded = tokenizer.decode(input_ids)
print(decoded)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

{'input_ids': [101, 58263, 10127, 10297, 26462, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
[101, 58263, 10127, 10297, 26462, 102]
[CLS] transformers is so cool [SEP]


In [2]:
from transformers import BertModel

model = BertModel.from_pretrained("google-bert/bert-base-multilingual-uncased")

for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

embeddings
└ word_embeddings
└ position_embeddings
└ token_type_embeddings
└ LayerNorm
└ dropout
encoder
└ layer
│  └ 0
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 1
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 2
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 3
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 4
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 5
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 6
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 7
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 8
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 9
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 10
│  │  └ attention
│  │  └ intermediate
│  │  └ output
│  └ 11
│  │  └ attention
│  │  └ intermediate
│  │  └ output
pooler
└ dense
└ activation


In [3]:
!pip install datasets  # 'datasets' 패키지 설치
from datasets import load_dataset  # 데이터셋을 로드하는 함수
from transformers import BertTokenizer, BertForSequenceClassification  # BERT 모델과 토크나이저 임포트

# 데이터 전처리 함수 정의
def preprocess_data(example, tokenizer):
    """
    각 데이터를 토큰화하여 BERT 모델에 맞는 형식으로 변환합니다.
    Args:
        example (dict): 데이터셋의 개별 예제.
        tokenizer (transformers.BertTokenizer): BERT 모델의 토크나이저.
    Returns:
        dict: 토큰화된 데이터.
    """
    return tokenizer(example["document"], truncation=True)

# BERT 모델과 토크나이저 초기화
model_name = "google-bert/bert-base-multilingual-uncased"  # 사용할 BERT 모델 이름
tokenizer = BertTokenizer.from_pretrained(model_name)  # 사전 학습된 토크나이저 불러오기
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2개 라벨 분류용 BERT 모델 불러오기

# 데이터셋 로드 및 전처리
dataset = load_dataset("nsmc", trust_remote_code=True)  # 'nsmc' 데이터셋 로드 (한국어 감정 분석 데이터)
processed_dataset = dataset.map(
    lambda example: preprocess_data(example, tokenizer),  # 데이터셋 각 예제를 토큰화하는 함수 적용
    batched=True,  # 배치 단위로 처리하여 속도 향상
    remove_columns=["id", "document"]  # 'id'와 'document' 열을 제거
).rename_column("label", "labels")  # 'label' 열 이름을 'labels'로 변경 (모델에 맞는 입력 형식)

# 데이터셋 확인
print(dataset)  # 원본 데이터셋 출력
print(processed_dataset)  # 전처리된 데이터셋 출력
print(dataset["train"][0])  # 원본 데이터셋의 첫 번째 훈련 데이터 예제 출력
print(processed_dataset["train"][0])  # 전처리된 데이터셋의 첫 번째 훈련 데이터 예제 출력

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})
DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})
{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
{'labels': 0, 'input_ids': [101, 1174, 25539, 23236, 29234, 13045, 119, 119, 87550, 97082, 25539, 1176, 25539, 24937, 13045, 16801, 72197, 47024, 1169, 70724, 22585, 13926, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [4]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

max_length_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="max_length"
)
max_length_dataloader = DataLoader(
    processed_dataset["train"],
    collate_fn=max_length_collator,
    batch_size=4,
    shuffle=False
)
max_length_iterator = iter(max_length_dataloader)
max_lnegth_batch = next(max_length_iterator)
print("max_length 패딩 입력 id shape :", max_lnegth_batch["input_ids"].shape)

longest_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest"
)
longest_dataloader = DataLoader(
    processed_dataset["train"],
    collate_fn=longest_collator,
    batch_size=4,
    shuffle=False
)
longest_iterator = iter(longest_dataloader)
longest_batch = next(longest_iterator)
print("longest 패딩 입력 id shape :", longest_batch["input_ids"].shape)

max_length 패딩 입력 id shape : torch.Size([4, 512])
longest 패딩 입력 id shape : torch.Size([4, 42])


In [5]:
from transformers import BertConfig, TrainingArguments, Trainer

# 드롭아웃 비율 조정
config = BertConfig.from_pretrained(model_name, num_labels=2, hidden_dropout_prob=0.3)
model = BertForSequenceClassification.from_pretrained(model_name, config=config)

# 에포크 수 수정
training_args = TrainingArguments(
    output_dir="text-classification",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,  # 에포크 수를 3으로 조정
    eval_steps=200,
    logging_steps=200,
    seed=42
)

# Trainer 객체 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=longest_collator,
    train_dataset=processed_dataset["train"].select(range(10000)),
    eval_dataset=processed_dataset["test"].select(range(100))
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
200,0.700700
400,0.697400
600,0.702600
800,0.700000
1000,0.699400
1200,0.698600
1400,0.698500
1600,0.696200
1800,0.699000
2000,0.702800


TrainOutput(global_step=3750, training_loss=0.6976614624023437, metrics={'train_runtime': 700.5975, 'train_samples_per_second': 42.821, 'train_steps_per_second': 5.353, 'total_flos': 1246768180264320.0, 'train_loss': 0.6976614624023437, 'epoch': 3.0})

In [6]:
!pip install evaluate
import evaluate

yhat = trainer.predict(processed_dataset["test"])
predictions = yhat.predictions.argmax(axis=1)
references = yhat.label_ids

metric = evaluate.load("accuracy")
accuracy = metric.compute(predictions=predictions, references=references)
print(accuracy)

metric = evaluate.load("f1")
f1 = metric.compute(predictions=predictions, references=references)
print(f1)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00


{'accuracy': 0.54968}


{'f1': 0.6793323459040674}


In [7]:
from transformers import BertConfig, TrainingArguments, Trainer

# 드롭아웃 비율 조정
config = BertConfig.from_pretrained(model_name, num_labels=2, hidden_dropout_prob=0.1)
model = BertForSequenceClassification.from_pretrained(model_name, config=config)

# 에포크 수 수정
training_args = TrainingArguments(
    output_dir="text-classification",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,  # 에포크 수를 3으로 조정
    eval_steps=200,
    logging_steps=200,
    seed=42
)


# Trainer 객체 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=longest_collator,
    train_dataset=processed_dataset["train"].select(range(10000)),
    eval_dataset=processed_dataset["test"].select(range(100))
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
200,0.664900
400,0.610200
600,0.619100
800,0.619900
1000,0.600700
1200,0.577300
1400,0.577500
1600,0.517500
1800,0.579300
2000,0.599600


TrainOutput(global_step=3750, training_loss=0.5246986246744791, metrics={'train_runtime': 680.216, 'train_samples_per_second': 44.104, 'train_steps_per_second': 5.513, 'total_flos': 1246768180264320.0, 'train_loss': 0.5246986246744791, 'epoch': 3.0})

In [8]:
!pip install evaluate
import evaluate

yhat = trainer.predict(processed_dataset["test"])
predictions = yhat.predictions.argmax(axis=1)
references = yhat.label_ids

metric = evaluate.load("accuracy")
accuracy = metric.compute(predictions=predictions, references=references)
print(accuracy)

metric = evaluate.load("f1")
f1 = metric.compute(predictions=predictions, references=references)
print(f1)

{'accuracy': 0.79338}
{'f1': 0.7943301945014035}
